In [ ]:
from typing import Any, Dict, Optional
import random
import time

# -----------------------------
# Step 0: Define Tools / Registry
# -----------------------------
TOOLS = {
    "weather_api": {"type": "api", "auth_required": True, "roles_allowed": ["user", "admin"]},
    "sales_db": {"type": "db", "auth_required": True, "roles_allowed": ["analyst", "admin"]},
    "deploy_tool": {"type": "action", "auth_required": True, "roles_allowed": ["admin"]},
}

# -----------------------------
# Step 1: Discover
# -----------------------------
def discover_tool(user_request: str) -> Optional[str]:
    """
    Decide which tool to use based on user request.
    """
    request = user_request.lower()
    if "weather" in request:
        return "weather_api"
    elif "sales data" in request:
        return "sales_db"
    elif "deploy" in request:
        return "deploy_tool"
    else:
        return None  # No tool needed

# -----------------------------
# Step 2: Authorize
# -----------------------------
def authorize_tool(tool_name: str, user_role: str) -> bool:
    """
    Check if user has permission to use the tool.
    """
    if tool_name not in TOOLS:
        return False
    tool = TOOLS[tool_name]
    return user_role in tool["roles_allowed"]

# -----------------------------
# Step 3: Execute
# -----------------------------
def execute_tool(tool_name: str, input_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Execute the tool and return a response.
    Simulate API/DB execution.
    """
    # Simulate random failures
    if random.random() < 0.2:
        raise RuntimeError("Tool failed or returned invalid response")

    if tool_name == "weather_api":
        return {"city": input_data["city"], "temperature": 28, "unit": input_data.get("unit", "C")}
    elif tool_name == "sales_db":
        return {"quarter": "Q4", "sales": 123456}
    elif tool_name == "deploy_tool":
        return {"status": "deployed", "service": input_data.get("service", "unknown")}
    else:
        return {"result": "default response"}

# -----------------------------
# Step 4: Fallback
# -----------------------------
def fallback_handler(tool_name: str, input_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle failure gracefully.
    """
    print(f"[Fallback] Handling failure for tool: {tool_name}")
    # Retry once
    try:
        time.sleep(1)  # simulate wait
        return execute_tool(tool_name, input_data)
    except Exception:
        # Return degraded response
        return {"message": "Unable to fetch data, returning best-effort response."}

# -----------------------------
# Agent Orchestration
# -----------------------------
def agent_process(user_request: str, user_role: str) -> Dict[str, Any]:
    """
    Full agent flow: Discover → Authorize → Execute → Fallback
    """
    print(f"[Agent] User request: {user_request}")

    # Discover
    tool = discover_tool(user_request)
    if not tool:
        return {"response": f"LLM handles internally: {user_request}"}

    # Authorize
    if not authorize_tool(tool, user_role):
        return {"response": f"Access denied for tool '{tool}' for role '{user_role}'"}

    # Execute with Fallback
    input_data = {"city": "Bangalore", "unit": "C", "service": "api_service"}  # example
    try:
        result = execute_tool(tool, input_data)
    except Exception as e:
        result = fallback_handler(tool, input_data)

    return {"response": result}

# -----------------------------
# Example Usage
# -----------------------------
if __name__ == "__main__":
    requests = [
        ("What is the weather in Bangalore?", "user"),
        ("Get sales data for last quarter", "analyst"),
        ("Deploy API service to production", "user"),
        ("Deploy API service to production", "admin")
    ]

    for req, role in requests:
        response = agent_process(req, role)
        print(f"Final Response: {response}\n")


[Agent] User request: What is the weather in Bangalore?
[Fallback] Handling failure for tool: weather_api
Final Response: {'response': {'city': 'Bangalore', 'temperature': 28, 'unit': 'C'}}

[Agent] User request: Get sales data for last quarter
Final Response: {'response': {'quarter': 'Q4', 'sales': 123456}}

[Agent] User request: Deploy API service to production
Final Response: {'response': "Access denied for tool 'deploy_tool' for role 'user'"}

[Agent] User request: Deploy API service to production
Final Response: {'response': {'status': 'deployed', 'service': 'api_service'}}

